In [8]:
using BenchmarkTools

In [9]:
function Convolution_2d(input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

function Convolution_2d!(ret, input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)

    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            ret[r, c] = sum(patch .* kernel) + bias
        end
    end
end

Convolution_2d! (generic function with 1 method)

In [10]:
function Conv_forward_v1(input, weights, bias)
    input_height, input_width, input_channels = size(input)
    kernel_height, kernel_width, _, output_channels = size(weights)

    output_height = input_height - kernel_height + 1
    output_width = input_width - kernel_width + 1
    output = zeros(Float32, output_height, output_width, output_channels)

    for k in 1:output_channels
        for c in 1:input_channels
            output[:, :, k] += Convolution_2d(input[:, :, c], weights[:, :, c, k]; bias=bias[k])
        end
    end
    
    return output
end

Conv_forward_v1 (generic function with 1 method)

In [11]:
function Conv_forward_v2(input, weights, bias)
    input_height, input_width, input_channels = size(input)
    kernel_height, kernel_width, _, output_channels = size(weights)

    output_height = input_height - kernel_height + 1
    output_width = input_width - kernel_width + 1
    output = zeros(Float32, output_height, output_width, output_channels)

    for k in 1:output_channels
        for c in 1:input_channels
            output[:, :, k] .+= Convolution_2d(input[:, :, c], weights[:, :, c, k]; bias=bias[k])
        end
    end
    
    return output
end

Conv_forward_v2 (generic function with 1 method)

In [12]:
function Conv_forward_v3(input, weights, bias)
    input_height, input_width, input_channels = size(input)
    kernel_height, kernel_width, _, output_channels = size(weights)

    output = zeros(Float32, input_height - kernel_height + 1, input_width - kernel_width + 1, output_channels)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    ret = zeros(Float32, 26, 26)
    for k in 1:output_channels
        for c in 1:input_channels
            tmp_input .= @views input[:, :, c]
            tmp_weights .= @views weights[:, :, c, k]
            Convolution_2d!(ret,tmp_input, tmp_weights; bias=bias[k])
            output[:, :, k] .+= ret
        end
    end
    return output
end

Conv_forward_v3 (generic function with 1 method)

In [13]:
input = rand(Float32, 28, 28, 6);
weights = rand(Float32, 3, 3, 6, 16);
bias = rand(Float32, 16);

In [14]:
@benchmark Conv_forward_v1(input, weights, bias)

BenchmarkTools.Trial: 951 samples with 1 evaluation.
 Range (min … max):  4.282 ms … 17.187 ms  ┊ GC (min … max): 0.00% … 10.01%
 Time  (median):     4.840 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.235 ms ±  1.232 ms  ┊ GC (mean ± σ):  3.58% ±  7.21%

  ▆▃▆█▄▂                                                      
  ████████▆█▇▇▅▄▄▄▄▃▃▃▄▂▃▃▃▃▃▂▂▃▂▃▂▃▁▂▁▂▂▂▂▂▂▁▂▁▁▂▁▂▂▁▂▂▁▁▁▂ ▃
  4.28 ms        Histogram: frequency by time        10.3 ms <

 Memory estimate: 7.08 MiB, allocs estimate: 65378.

In [15]:
@benchmark Conv_forward_v2(input, weights, bias)

BenchmarkTools.Trial: 1049 samples with 1 evaluation.
 Range (min … max):  4.214 ms …   9.673 ms  ┊ GC (min … max): 0.00% … 9.98%
 Time  (median):     4.461 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.751 ms ± 742.334 μs  ┊ GC (mean ± σ):  2.33% ± 4.88%

  ██▅▅▄▃▃▄▂▄▄▃▂▁ ▁    ▁ ▁                                      
  █████████████████████▇█▆▆▇█▇▇▅▇▆▅▇▄▄▄▅▁▆▁▆▆▇▅▅▅▅▄▁▄▅▁▄▁▄▁▄▅ █
  4.21 ms      Histogram: log(frequency) by time      7.93 ms <

 Memory estimate: 6.82 MiB, allocs estimate: 65282.

In [16]:
@benchmark Conv_forward_v3(input, weights, bias)

BenchmarkTools.Trial: 1114 samples with 1 evaluation.
 Range (min … max):  4.156 ms …   7.883 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.351 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.476 ms ± 378.723 μs  ┊ GC (mean ± σ):  2.29% ± 4.89%

  █▄▆▂                                                         
  ████▇▅▅▅▆▅▅▅▅▄▄▅▄▄▃▃▃▃▄▄▄▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▃▂▂▂▂▂▃▂▂▂▂▂▂▂▂▁▁▂▂ ▃
  4.16 ms         Histogram: frequency by time        5.71 ms <

 Memory estimate: 6.52 MiB, allocs estimate: 65093.

In [20]:
Conv_forward_v2(input, weights, bias) == Conv_forward_v3(input, weights, bias)

true